# Adaptive

In [ ]:
import adaptive
adaptive.notebook_extension()

def func(x, wait=True):
    """Function with a sharp peak on a smooth background"""
    import numpy as np
    from time import sleep
    from random import randint

    x = np.asarray(x)
    a = 0.001
    if wait:
        sleep(np.random.randint(0, 2) / 10)
    return x + a**2/(a**2 + (x)**2)

## Local Process Pool (default)

In [ ]:
learner = adaptive.learner.Learner1D(func, bounds=(-1.01, 1.0))
runner = adaptive.Runner(learner, goal=lambda l: l.loss(real=True) < 0.01)
adaptive.live_plot(runner)

In [ ]:
# Same function evaluated on homogeneous grid with same amount of points
from functools import partial
import numpy as np

learner2 = adaptive.learner.Learner1D(func, bounds=(-1.01, 1.0))
xs = np.linspace(-1.01, 1.0, len(learner.data))
learner2.add_data(xs, map(partial(func, wait=False), xs))
learner2.plot()

## ipyparallel

In [ ]:
import ipyparallel

client = ipyparallel.Client()

# Initialize the learner
learner = adaptive.learner.Learner1D(func)
runner = adaptive.Runner(learner, client, goal=lambda l: l.loss() < 0.1)
adaptive.live_plot(runner)

## 0D Learner

In [ ]:
def func(x):
    import random
    import numpy as np
    from time import sleep
    sleep(np.random.randint(0, 2))
    return random.gauss(0.05, 1)

learner = adaptive.learner.AverageLearner(func, None, 0.1)
runner = adaptive.Runner(learner, goal=lambda l: l.loss() < 1)